<a href="https://colab.research.google.com/github/jagalindo/UVL-ggnf-grammar/blob/main/LLAMA2_CPP_grammars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python
!pip install httpx
!wget https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_0.gguf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 56.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.35-cp310-cp310-manylinux_2_35_x86_64.whl size=2461136 sha256=14d757fc51b77b5f985b9e8e35dc1a3606483d2d108b7a6a280484e95391c672
  Stored in directory: /root/.cache/pip/wheels/fc/94/3a/e2167159b1e2e06e90f352e0111a15f09cd1fb403eaab61fd2
Successfully built llama-cpp-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00

In [15]:
from llama_cpp import Llama, LlamaGrammar
import httpx

grammar_text = httpx.get("https://raw.githubusercontent.com/ggerganov/llama.cpp/master/grammars/json_arr.gbnf").text
grammar = LlamaGrammar.from_string(grammar_text)

template = """You are a helpful assistant. Help the user answer any questions.

Example 1:

Human: Sandwich

Assistant:

features
    Sandwich
        mandatory
            Bread
        optional
            Sauce
                alternative
                    Ketchup
                    Mustard
            Cheese
Begin!"""


llm = Llama(
model_path="/content/llama-2-7b-chat.Q5_0.gguf",
#grammar_path="/content/uvl.gguf",
temperature=0.75,
max_tokens=500,
top_p=1,
verbose=True,
n_gpu_layers=-1,
# seed=1337, # Uncomment to set a specific seed
# n_ctx=2048, # Uncomment to increase the context window
)

output = llm(
      "Pizza", # Prompt
      max_tokens=500, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      #stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=True, # Echo the prompt back in the output
      grammar=grammar
)

print(output)

from_string grammar:
root ::= arr 
arr ::= [[] [<U+000A>] ws arr_12 []] 
value ::= object | array | string | number | value_7 ws 
object ::= [{] ws object_16 [}] ws 
array ::= [[] ws array_20 []] ws 
string ::= ["] string_23 ["] ws 
number ::= number_24 number_30 number_34 ws 
value_7 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_36 
arr_9 ::= value arr_11 
arr_10 ::= [,] [<U+000A>] ws value 
arr_11 ::= arr_10 arr_11 | 
arr_12 ::= arr_9 | 
object_13 ::= string [:] ws value object_15 
object_14 ::= [,] ws string [:] ws value 
object_15 ::= object_14 object_15 | 
object_16 ::= object_13 | 
array_17 ::= value array_19 
array_18 ::= [,] ws value 
array_19 ::= array_18 array_19 | 
array_20 ::= array_17 | 
string_21 ::= [^"\] | [\] string_22 
string_22 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_23 ::= string_21 string_23 | 
number_24 ::= number_25 number_26 
number_25 ::= [-] | 
number_26 ::= [0-9] | [1-9] number_27 
number_27 ::= [0

{'id': 'cmpl-5a2915d1-6855-43d9-b216-7a895b225263', 'object': 'text_completion', 'created': 1706532133, 'model': '/content/llama-2-7b-chat.Q5_0.gguf', 'choices': [{'text': 'Pizza[\n\n\n1.2 \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

In [16]:
import json
print(json.dumps(json.loads(output['choices'][0]['text']), indent=4))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)